# CITS 影像分析與能帶/能譜圖測試 (簡化版) / CITS Image Analysis with Band/Spectrum Testing (Simplified)

**作者 / Author**: Odindino  
**測試目標**:

1. 📂 載入 CITS 檔案
2. 🎛️ 查看不同偏壓切片
3. 📍 選擇點位生成能譜圖 (Energy Spectrum)
4. 📈 拉線生成能帶圖 (Band Profile)
5. 🧮 計算和顯示 dI/dV 圖

**使用方式**: 直接修改程式碼中的設定，執行 cell 來測試

## 📦 模組載入 / Module Loading

In [4]:
import sys
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import plotly.io as pio

# 設置 Plotly
pyo.init_notebook_mode(connected=True)
pio.renderers.default = "notebook"

# 添加後端路徑
backend_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

# 導入 KEEN 模組
from core.experiment_session import ExperimentSession

print("✅ 模組載入完成")
print(f"📂 工作目錄: {os.getcwd()}")
print(f"🐍 後端路徑: {backend_path}")

✅ 模組載入完成
📂 工作目錄: /Users/yangziliang/Git-Projects/keen/backend/test/notebooks
🐍 後端路徑: /Users/yangziliang/Git-Projects/keen/backend


## ⚙️ 設定區域 / Configuration Area

### 🔧 在這裡修改您的設定 / Modify your settings here

In [5]:
# ============================================================================
# 📝 請在這裡修改您的設定 / MODIFY YOUR SETTINGS HERE
# ============================================================================

# 📂 檔案路徑設定 / File Path Settings
TXT_FILE_PATH = '../../../testfile/20250521_Janus Stacking SiO2_13K_113.txt'

# 🎛️ 偏壓設定 / Bias Settings
BIAS_INDEX = 50  # 👈 在這裡修改要顯示的偏壓索引 (0 開始)

# 📍 單點能譜座標設定 / Point Spectrum Coordinates
POINT_X = 50    # 👈 點能譜 X 座標
POINT_Y = 50    # 👈 點能譜 Y 座標

# 📈 能帶剖面線設定 / Band Profile Line Settings
# 格式: (y, x) - 注意是 y 在前!
LINE_START = (25, 10)   # 👈 起點座標 (y, x)
LINE_END = (75, 90)     # 👈 終點座標 (y, x)

# 🧮 dI/dV 計算設定 / dI/dV Calculation Settings
DIDV_BIAS_INDEX = 50    # 👈 要顯示 dI/dV 的偏壓索引

# ============================================================================
# 📋 設定檢查 / Settings Check
# ============================================================================

print("🔧 當前設定 / Current Settings:")
print(f"📂 TXT 檔案: {TXT_FILE_PATH}")
print(f"🎛️ 偏壓索引: {BIAS_INDEX}")
print(f"📍 點能譜座標: ({POINT_X}, {POINT_Y})")
print(f"📈 能帶剖面: {LINE_START} → {LINE_END}")
print(f"🧮 dI/dV 偏壓索引: {DIDV_BIAS_INDEX}")
print()
print(f"📁 TXT 檔案存在: {os.path.exists(TXT_FILE_PATH)}")

# 全局變數初始化
session = None
cits_data = None

🔧 當前設定 / Current Settings:
📂 TXT 檔案: ../../../testfile/20250521_Janus Stacking SiO2_13K_113.txt
🎛️ 偏壓索引: 50
📍 點能譜座標: (50, 50)
📈 能帶剖面: (25, 10) → (75, 90)
🧮 dI/dV 偏壓索引: 50

📁 TXT 檔案存在: True


## 📂 步驟 1: 載入檔案 / Step 1: Load Files

In [6]:
def load_cits_files():
    """載入 CITS 檔案"""
    global session, cits_data
    
    try:
        print("📂 開始載入檔案...")
        
        # 初始化會話
        session = ExperimentSession(TXT_FILE_PATH)
        print(f"✅ 實驗載入成功: {session.experiment_name}")
        
        # 列出可用的 CITS 檔案
        cits_files = session.get_cits_files()
        print(f"\n📋 可用的 CITS 檔案: {cits_files}")
        
        if not cits_files:
            print("❌ 沒有找到 CITS 檔案")
            return False
            
        # 載入第一個 CITS 檔案
        cits_key = cits_files[0]
        print(f"\n🔄 載入 CITS 檔案: {cits_key}")
        
        cits_proxy = session.get_file(cits_key)
        cits_data = cits_proxy.data
        
        # 顯示 CITS 資訊
        print(f"✅ CITS 檔案載入成功")
        print(f"📊 資料形狀: {cits_data.cits_data.shape} (height, width, bias)")
        print(f"📏 掃描範圍: {cits_data.x_range:.1f} × {cits_data.y_range:.1f} nm")
        print(f"⚡ 偏壓範圍: {cits_data.bias_range[0]:.3f} ~ {cits_data.bias_range[-1]:.3f} V")
        print(f"⚡ 偏壓點數: {len(cits_data.bias_range)}")
        
        return True
        
    except Exception as e:
        print(f"❌ 載入失敗: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return False

# 執行載入
load_success = load_cits_files()
if load_success:
    print("\n🎉 CITS 檔案載入成功! 繼續下一步。")
    
    # 檢查設定的索引是否在範圍內
    max_bias_index = len(cits_data.bias_range) - 1
    h, w, _ = cits_data.cits_data.shape
    
    print(f"\n📏 資料範圍檢查:")
    print(f"   偏壓索引範圍: 0 ~ {max_bias_index}")
    print(f"   座標範圍: X (0 ~ {w-1}), Y (0 ~ {h-1})")
    
    # 檢查設定值
    if BIAS_INDEX > max_bias_index:
        print(f"⚠️  警告: 偏壓索引 {BIAS_INDEX} 超出範圍，請修改為 0~{max_bias_index}")
    if POINT_X >= w or POINT_Y >= h:
        print(f"⚠️  警告: 點座標 ({POINT_X}, {POINT_Y}) 超出範圍")
    if LINE_START[0] >= h or LINE_START[1] >= w or LINE_END[0] >= h or LINE_END[1] >= w:
        print(f"⚠️  警告: 線座標超出範圍")
        
else:
    print("\n💡 請檢查檔案路徑")

📂 開始載入檔案...
✅ 實驗載入成功: Unknown

📋 可用的 CITS 檔案: ['20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix', '20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix']

🔄 載入 CITS 檔案: 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix
✅ CITS 檔案載入成功
❌ 載入失敗: 'CitsData' object has no attribute 'cits_data'
Traceback (most recent call last):
  File "/var/folders/s8/x7qjkqzs44lg2tdb36y2xh680000gn/T/ipykernel_98477/130811388.py", line 29, in load_cits_files
    print(f"📊 資料形狀: {cits_data.cits_data.shape} (height, width, bias)")
                          ^^^^^^^^^^^^^^^^^^^
AttributeError: 'CitsData' object has no attribute 'cits_data'


💡 請檢查檔案路徑


/Users/yangziliang/Git-Projects/keen/backend/core/parsers/dat_parser.py:84: DtypeWarning:

Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.



## 🎛️ 步驟 2: 顯示偏壓切片 / Step 2: Display Bias Slice

In [ ]:
def display_bias_slice(bias_index):
    """顯示指定偏壓的切片"""
    if cits_data is None:
        print("❌ 請先載入 CITS 檔案")
        return
        
    try:
        # 檢查索引範圍
        max_index = len(cits_data.bias_range) - 1
        if bias_index > max_index:
            print(f"❌ 偏壓索引 {bias_index} 超出範圍 (0~{max_index})")
            return
            
        # 獲取切片數據
        slice_data = cits_data.cits_data[:, :, bias_index]
        bias_value = cits_data.bias_range[bias_index]
        
        # 創建圖表
        fig = go.Figure()
        
        # 添加熱圖
        fig.add_trace(go.Heatmap(
            z=slice_data,
            colorscale='RdBu_r',
            colorbar=dict(title='Current (pA)'),
            hovertemplate='X: %{x}<br>Y: %{y}<br>Current: %{z:.3f} pA<extra></extra>'
        ))
        
        fig.update_layout(
            title=f'CITS 偏壓切片 @ {bias_value:.3f} V (Index: {bias_index})',
            xaxis_title='X (pixels)',
            yaxis_title='Y (pixels)',
            width=600,
            height=600,
            yaxis=dict(scaleanchor="x", scaleratio=1)
        )
        
        fig.show()
        
        # 顯示統計
        print(f"\n📊 切片統計 (@ {bias_value:.3f} V):")
        print(f"   電流範圍: {np.min(slice_data):.3f} ~ {np.max(slice_data):.3f} pA")
        print(f"   平均電流: {np.mean(slice_data):.3f} pA")
        print(f"   標準差: {np.std(slice_data):.3f} pA")
        
    except Exception as e:
        print(f"❌ 顯示失敗: {str(e)}")

# 顯示指定的偏壓切片
if load_success:
    print(f"🎛️ 顯示偏壓切片 (索引: {BIAS_INDEX}):")
    display_bias_slice(BIAS_INDEX)
else:
    print("⚠️ 請先成功載入 CITS 檔案")

## 📋 測試總結 / Test Summary

In [ ]:
print("📋 CITS 分析測試總結")
print("=" * 50)

if load_success and cits_data is not None:
    print(f"✅ CITS 檔案載入成功")
    print(f"📊 資料維度: {cits_data.cits_data.shape}")
    print(f"⚡ 偏壓範圍: {cits_data.bias_range[0]:.3f} ~ {cits_data.bias_range[-1]:.3f} V")
    print(f"📏 空間範圍: {cits_data.x_range:.1f} × {cits_data.y_range:.1f} nm")
    print()
    print("🔧 可用功能 (可在新 cell 中測試):")
    print("   1. 修改 BIAS_INDEX 並重新執行步驟 2")
    print("   2. 添加點能譜分析功能")
    print("   3. 添加能帶剖面分析功能")
    print("   4. 添加 dI/dV 計算功能")
else:
    print("❌ CITS 檔案載入失敗")
    
print("\n💡 使用提示:")
print("1. 修改設定區域的參數後重新執行對應的 cell")
print("2. 如需添加更多功能，可參考完整版筆記本")
print("3. 所有座標使用像素單位 (0 開始)")